In [14]:
import math
from itertools import combinations
import numpy as np
import random
from Problem import Problem

GOAL
Cost minimization

## Simple Test Problem

In [2]:
CITIES = [
    "Rome",
    "Milan",
    "Naples",
    "Turin",
    "Palermo",
    "Genoa",
    "Bologna",
    "Florence",
    "Bari",
    "Catania",
    "Venice",
    "Verona",
    "Messina",
    "Padua",
    "Trieste",
    "Taranto",
    "Brescia",
    "Prato",
    "Parma",
    "Modena",
]
test_problem = np.load('problems/test_problem.npy')

### LOAD PROBLEMS

In [3]:
g_10 = Problem("g_10", np.load('problems/problem_g_10.npy'))
g_20 = Problem("g_20", np.load('problems/problem_g_20.npy'))
g_50 = Problem("g_50", np.load('problems/problem_g_50.npy'))
g_100 = Problem("g_100", np.load('problems/problem_g_100.npy'))
g_200 = Problem("g_200", np.load('problems/problem_g_200.npy'))
g_500 = Problem("g_500", np.load('problems/problem_g_500.npy'))
g_1000 = Problem("g_1000", np.load('problems/problem_g_1000.npy'))

r1_10 = Problem("r1_10", np.load('problems/problem_r1_10.npy'))
r1_20 = Problem("r1_20", np.load('problems/problem_r1_20.npy'))
r1_50 = Problem("r1_50", np.load('problems/problem_r1_50.npy'))
r1_100 = Problem("r1_100", np.load('problems/problem_r1_100.npy'))
r1_200 = Problem("r1_200", np.load('problems/problem_r1_200.npy'))
r1_500 = Problem("r1_500", np.load('problems/problem_r1_500.npy'))
r1_1000 = Problem("r1_1000", np.load('problems/problem_r1_1000.npy'))

r2_10 = Problem("r2_10", np.load('problems/problem_r2_10.npy'))
r2_20 = Problem("r2_20", np.load('problems/problem_r2_20.npy'))
r2_50 = Problem("r2_50", np.load('problems/problem_r2_50.npy'))
r2_100 = Problem("r2_100", np.load('problems/problem_r2_100.npy'))
r2_200 = Problem("r2_200", np.load('problems/problem_r2_200.npy'))
r2_500 = Problem("r2_500", np.load('problems/problem_r2_500.npy'))
r2_1000 = Problem("r2_1000", np.load('problems/problem_r2_1000.npy'))



# DEFINE ARRAY FOR HANDLE ALL THE PROBLEMS

problems_arr = [g_10, g_20, g_50, g_100, g_200, g_500, g_1000,
                r1_10, r1_20, r1_50, r1_100, r1_200, r1_500, r1_1000,
                r2_10, r2_20, r2_50, r2_100, r2_200, r2_500, r2_1000]

### GENERAL UTILS FUNCTIONS

In [4]:
def check_negative_value(distance_matrix):
    # Negative values?
    return np.any(distance_matrix < 0)


def check_diagonal_all_zero(distance_matrix):
    # Diagonal is all zero?
    return np.allclose(np.diag(distance_matrix), 0.0)


def check_symmetry(distance_matrix):
    # Symmetric matrix?
    return np.allclose(distance_matrix, distance_matrix.T)


def check_triangular_inequality(distance_matrix):
    # Triangular inequality
    return all(
        distance_matrix[x, y] <= distance_matrix[x, z] + distance_matrix[z, y]
        for x, y, z in list(combinations(range(distance_matrix.shape[0]), 3))
    )

In [5]:
print(f"check negative value | g_100: {check_negative_value(g_100.distance_matrix)}")
print(f"check diagonal all 0s | g_100: {check_diagonal_all_zero(g_100.distance_matrix)}")
print(f"check symmetry | g_100: {check_symmetry(g_100.distance_matrix)}")
print(f"check triangular inequality | g_100 {check_triangular_inequality(g_100.distance_matrix)}")
print()
print(f"check negative value | r1_100: {check_negative_value(r1_100.distance_matrix)}")
print(f"check diagonal all 0s | r1_100: {check_diagonal_all_zero(r1_100.distance_matrix)}")
print(f"check symmetry | r1_100: {check_symmetry(r1_100.distance_matrix)}")
print(f"check triangular inequality | r1_100 {check_triangular_inequality(r1_100.distance_matrix)}")
print()
print(f"check negative value | r2_100: {check_negative_value(r2_100.distance_matrix)}")
print(f"check diagonal all 0s | r2_100: {check_diagonal_all_zero(r2_100.distance_matrix)}")
print(f"check symmetry | r2_100: {check_symmetry(r2_100.distance_matrix)}")
print(f"check triangular inequality | r2_100 {check_triangular_inequality(r2_100.distance_matrix)}")
print()

check negative value | g_100: False
check diagonal all 0s | g_100: True
check symmetry | g_100: True
check triangular inequality | g_100 True

check negative value | r1_100: False
check diagonal all 0s | r1_100: True
check symmetry | r1_100: False
check triangular inequality | r1_100 False

check negative value | r2_100: True
check diagonal all 0s | r2_100: False
check symmetry | r2_100: False
check triangular inequality | r2_100 False



In [6]:
# HYPERPARAMETERS

POPULATION_SIZE = 100
NUM_GENERATIONS = 1000
ELITISM_SIZE = 2
TOURNAMENT_K = 3
CROSSOVER_RATE = 0.9
MUTATION_RATE = 0.2

In [7]:
def create_initial_population(pop_size, num_cities):
    """Creates a list of random tours (solutions)."""
    population = []
    for _ in range(pop_size):
        tour = np.random.permutation(num_cities)
        population.append(tour)
    return population

In [8]:
def calculate_fitness(tour, distance_matrix):
    """
    Calculates the total cost (fitness) of a single tour.
    This is the value we want to MINIMIZE.
    Works for both symmetric and asymmetric matrices.
    """
    total_cost = 0
    num_cities = len(tour)
    for i in range(num_cities):
        city_a = tour[i]
        # Use modulo (%) to wrap around to the start city
        city_b = tour[(i + 1) % num_cities]
        
        # This correctly takes the cost from A -> B
        total_cost += distance_matrix[city_a, city_b]
        
    return total_cost

In [9]:
def tournament_selection(population, fitness_scores, k):
    """
    Selects one parent using k-tournament selection.    
    Picks k random individuals and returns the fittest one.
    """
    indices = np.random.choice(len(population), k, replace=False)
    k_fitnesses = [fitness_scores[i] for i in indices]
    best_local_idx = np.argmin(k_fitnesses)
    best_global_idx = indices[best_local_idx]
    
    return population[best_global_idx]

In [10]:
def crossover(parent1, parent2):
    """
    Perform crossover
    """
    num_cities = len(parent1)
    offspring = np.full(num_cities, -1)
    
    # Select two random cut points
    start, end = sorted(np.random.choice(num_cities, 2, replace=False))
    
    # Copy the slice from parent1
    offspring[start:end] = parent1[start:end]
    
    # Fill remaining slots from parent2
    parent2_ptr = 0
    offspring_ptr = 0
    cities_in_offspring = set(offspring[start:end])
    
    while -1 in offspring:
        # Find next available slot in offspring
        if offspring_ptr == start:
            offspring_ptr = end
            
        # Get city from parent2
        city_to_add = parent2[parent2_ptr]
        parent2_ptr += 1
        
        if city_to_add not in cities_in_offspring:
            offspring[offspring_ptr] = city_to_add
            offspring_ptr += 1
            
    return offspring

In [11]:
# DIFFERENT KINDS OF MUTATIONS


def swap_mutation(tour):
    """Performs a simple swap mutation on a tour (modifies in-place)."""
    idx1, idx2 = np.random.choice(len(tour), 2, replace=False)
    tour[idx1], tour[idx2] = tour[idx2], tour[idx1]


def inversion_mutation(tour):
    """
    Performs an inversion mutation (modifies in-place).
    """
    # Select two random cut points
    start, end = sorted(np.random.choice(len(tour), 2, replace=False))
    if start >= end:
        return 
    
    # Reverse the sub-sequence in-place
    tour[start:end] = tour[start:end][::-1]

In [12]:
def algo_1(problem: Problem, pop_size, num_generations, elitism_size, k, crossover_rate, mutation_rate, show_process = False):
    """
    Runs the main evolutionary algorithm loop.
    """

    distance_matrix = problem.distance_matrix
    num_cities = distance_matrix.shape[0]
    
    # Initializing
    population = create_initial_population(pop_size, num_cities)
    best_tour_so_far = None
    best_fitness_so_far = float('inf')
    
    fitness_history = []

    # Conditional print, implemented just for style
    if not show_process:
        print(f"{problem.name} | Running...")

    for gen in range(num_generations):
        # Calculate fitness for the entire population
        fitness_scores = [calculate_fitness(tour, distance_matrix) for tour in population]
        
        # Find the indices of the N best individuals,
        # The first index is the one that refers to the individual with the best fitness
        elite_indices = np.argsort(fitness_scores)[:elitism_size]
        # creating a new population composed of the best individuals of the previous
        new_population = [population[i].copy() for i in elite_indices]
        
        # Track the best-ever solution
        best_current_idx = elite_indices[0] 
        current_best_fitness = fitness_scores[best_current_idx]
        
        if current_best_fitness < best_fitness_so_far:
            best_fitness_so_far = current_best_fitness
            best_tour_so_far = population[best_current_idx].copy()

            # If True the new value computed will be printed, False otherwise
            if show_process:
                print(f"{problem.name} | Gen {gen}: New best cost = {best_fitness_so_far:.2f}")

        # Add the best cost of this generation to the history
        fitness_history.append(current_best_fitness)

        # In this loop we compute the generation
        while len(new_population) < pop_size:
            # Selecting 2 parents using tournament selection
            parent1 = tournament_selection(population, fitness_scores, k)
            parent2 = tournament_selection(population, fitness_scores, k)
            
            # performing the cross over (random decision defined by crossover_rate)
            if random.random() < crossover_rate:
                offspring = crossover(parent1, parent2)
            else:
                offspring = parent1.copy() 
            
            # performing the mutation
            # (again, its a random decision that doesn't even depend on whether the crossover is done or not)
            if random.random() < mutation_rate:
                # Here e we can choose to use inversion_mutation() and swap_mutation()
                inversion_mutation(offspring)
                
            new_population.append(offspring)

        # Replace the old population with the new one
        population = new_population

    return best_tour_so_far, best_fitness_so_far, fitness_history

In [ ]:
# DO THE COMPUTATION FOR ALL THE PROBLEMS ALL AT ONCE




for problem in problems_arr:
    best_tour, best_fitness, _ = algo_1(
        problem=problem,
        pop_size=POPULATION_SIZE,
        num_generations=NUM_GENERATIONS,
        elitism_size=ELITISM_SIZE,
        k=TOURNAMENT_K,
        crossover_rate=CROSSOVER_RATE,
        mutation_rate=MUTATION_RATE,
        show_process=False
    )
    # print(f"{problem.name} | Best tour: {best_tour}")
    print(f"{problem.name} | Total fitness: {best_fitness:.2f}\n")